In [11]:
import pandas as pd
import re
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Jack
[nltk_data]     D\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [7]:
test = pd.read_csv("Corona_NLP_test.csv",encoding='latin1')
train = pd.read_csv("Corona_NLP_train.csv",encoding='latin1')

In [8]:
test

,OriginalTweet,Sentiment,LowerTweet,BooleanSentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,trending: new yorkers encounter empty supermar...,0
1,When I couldn't find hand sanitizer at Fred Me...,Positive,when i couldn't find hand sanitizer at fred me...,1
2,Find out how you can protect yourself and love...,Extremely Positive,find out how you can protect yourself and love...,1
3,#Panic buying hits #NewYork City as anxious sh...,Negative,#panic buying hits #newyork city as anxious sh...,0
4,Voting in the age of #coronavirus = hand sanit...,Positive,voting in the age of #coronavirus = hand sanit...,1
...,...,...,...,...
3174,"@RicePolitics @MDCounties Craig, will you call...",Negative,"@ricepolitics @mdcounties craig, will you call...",0
3175,Meanwhile In A Supermarket in Israel -- People...,Positive,meanwhile in a supermarket in israel -- people...,1
3176,Did you panic buy a lot of non-perishable item...,Negative,did you panic buy a lot of non-perishable item...,0
3177,Gov need to do somethings instead of biar je r...,Extremely Negative,gov need to do somethings instead of biar je r...,0


In [9]:
def sub(text):
    temp = re.sub('[^a-zA-Z]', ' ', text )
    temp = re.sub(r'\s+', ' ', temp)
    return temp

test['LowerTweet'] = test['LowerTweet'].apply(sub)
train['LowerTweet'] = train['LowerTweet'].apply(sub)

In [14]:
AllWords = ''
for i in train['LowerTweet']:
    AllWords += i
print(AllWords)

all_sentences = nltk.sent_tokenize(AllWords)

all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
# Need to make word2vec standard across all our projects!

word2vec = Word2Vec(all_words, min_count=2)

vocabulary = word2vec.wv.key_to_index
print(vocabulary)

{'the': 0, 'to': 1, 't': 2, 'and': 3, 'of': 4, 'co': 5, 'https': 6, 'covid': 7, 'a': 8, 'in': 9, 'coronavirus': 10, 'for': 11, 'is': 12, 'are': 13, 'i': 14, 'you': 15, 'on': 16, 's': 17, 'this': 18, 'prices': 19, 'at': 20, 'it': 21, 'food': 22, 'we': 23, 'that': 24, 'with': 25, 'supermarket': 26, 'store': 27, 'have': 28, 'as': 29, 'be': 30, 'grocery': 31, 'people': 32, 'amp': 33, 'from': 34, 'all': 35, 'can': 36, 'not': 37, 'your': 38, 'will': 39, 'they': 40, 'consumer': 41, 'our': 42, 'my': 43, 'out': 44, 'or': 45, 'has': 46, 'up': 47, 'by': 48, 'but': 49, 'more': 50, 'their': 51, 'no': 52, 'shopping': 53, 'if': 54, 'online': 55, 'during': 56, 'so': 57, 'pandemic': 58, 'now': 59, 'who': 60, 'how': 61, 'get': 62, 'panic': 63, 'about': 64, 'workers': 65, 'need': 66, 'like': 67, 'what': 68, 'demand': 69, 'sanitizer': 70, 'us': 71, 'there': 72, 'time': 73, 'hand': 74, 'do': 75, 'just': 76, 'help': 77, 'was': 78, 'an': 79, 'home': 80, 'm': 81, 'go': 82, 'some': 83, 'when': 84, 're': 85, 'd